## Note
Some k-cross validation used when trying out kaggle bnank churn

In [ ]:
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

input_shape = loaded_data.shape[1:]
input_shape



def build_model():
  inputs = layers.Input(shape=input_shape)
  hidden = layers.Dense(40, activation='relu')(inputs)
  hidden = layers.Dropout(0.2)(hidden)
  hidden = layers.Dense(10, activation='relu')(hidden)
  hidden = layers.Dropout(0.2)(hidden) 
  outputs = layers.Dense(1)(hidden)

  model = keras.models.Model(inputs=inputs, outputs=outputs) 

  model.compile(optimizer=RMSprop(),
                loss=BinaryCrossentropy(),
                metrics=[keras.metrics.BinaryAccuracy()])
  
  return model

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        verbose=0,
    ),
    keras.callbacks.ModelCheckpoint(
        '/kaggle/working/models/best_model.keras', 
        monitor='accuracy',
        save_best_only=True
    )
]

cv = KFold(n_splits=6, shuffle=True, random_state=42)
data = loaded_data.to_numpy()
labels = loaded_labels.to_numpy()

fold_num = 1
all_accuracy_histories = []
all_loss_histories = []

for train_i, val_i in cv.split(data, labels):
  print(f"Processing fold #{fold_num}")
  scaler = StandardScaler()
  train_data = data[train_i]
  val_data = data[val_i]
  scaler.fit(train_data)
  train_data = scaler.transform(train_data)
  val_data = scaler.transform(val_data)

  model = build_model()

  if (fold_num == 1):
    model.summary()

  history = model.fit(train_data,
                      labels[train_i],
                      epochs=10,
                      batch_size = 128,
                      validation_data=(val_data, labels[val_i]),
                      callbacks=callbacks)


  # accuracy_history = history.history["val_accuracy"]
  # loss_history = history.history["val_loss"]
  # all_accuracy_histories.append(accuracy_history)
  # all_loss_histories.append(loss_history)
  fold_num += 1

# average_accuracy_history = [
#     np.mean([x[i] for x in all_accuracy_histories]) for i in range(10)]

# plt.plot(range(1, len(average_accuracy_history) + 1), average_accuracy_history)
# plt.xlabel("Epochs")
# plt.ylabel("Validation Accuracy")
# plt.show()

# average_loss_history = [
#     np.mean([x[i] for x in all_loss_histories]) for i in range(10)]

# plt.plot(range(1, len(average_loss_history) + 1), average_loss_history)
# plt.xlabel("Epochs")
# plt.ylabel("Validation Loss")
# plt.show()

  

